# K-Nearest Neighbors (KNN) - No-Framework Implementation

Multi-class classification on the **Covertype (Forest Cover Type)** dataset using pure NumPy.

**Dataset**: 581,012 samples, 54 features, 7 forest cover types  
**Task**: Predict forest cover type from cartographic variables  
**Key Concept**: KNN is a "lazy learner" - no training phase, expensive at prediction time

## What We'll Build From Scratch
- **Euclidean/Manhattan distance** calculation (vectorized)
- **K-nearest neighbor search** using argsort
- **Majority voting** for multi-class prediction
- **Batched prediction** to handle 464K training samples without OOM
- **Distance-weighted voting** (optional advanced feature)


In [1]:
# Standard libraries
import numpy as np
import sys

# Add utils to path
sys.path.append('../..')
from utils.data_loader import load_processed_data
from utils.metrics import accuracy, macro_f1_score, confusion_matrix_multiclass
from utils.visualization import (
    plot_confusion_matrix_multiclass,
    plot_validation_curve,
    plot_per_class_f1
)
from utils.performance import track_performance

print("Imports complete!")

Imports complete!


In [2]:
# Load Preprocessed data
"""
Data was preprocessed in data-preperation/preprocess_knn.py
    - 80/20 split
    - StandardScaler applied (fit on train only)
    - All 4 frameworks load identical data for fair comparison
"""

X_train, X_test, y_train, y_test, metadata = load_processed_data('knn')

# Extract metadata for reference
class_names = metadata['class_names']
n_classes = metadata['n_classes']

print(f"Training set: {X_train.shape[0]:,} samples, {X_train.shape[1]} features")
print(f"Test set: {X_test.shape[0]:,} samples")
print(f"Classes ({n_classes}): {class_names}")

Training set: 464,809 samples, 54 features
Test set: 116,203 samples
Classes (7): ['Spruce/Fir', 'Lodgepole Pine', 'Ponderosa Pine', 'Cottonwood/Willow', 'Aspen', 'Douglas-fir', 'Krummholz']


## Manhattan Distance Function

- Scikit-Learn found that **Manhattan distance** (L1) outperforms Euclidean (L2) for this dataset.
- We'll implement Manhattan distance from scratch.

**Manhattan Distance Formula.**
$$d(x, y) = \sum_{i=1}^{n} |x_i - y_i|$$

Sum of absolute differences across all features.

In [3]:
# Manhattan Distance (Vectorized)

def manhattan_distance_batch(X_batch, X_train):
    """
    Compute Manhattan (L1) distance from each sample in X_batch
    to all samples in X_train.

    Vectorized using broadcasting for efficiency:
        - X_batch: (batch_size, n_features)
        - X_train: (n_train, n_features)
        - Returns: (batch_size, n_train) distance matrix

    Broadcasting expands X_batch to (batch_size, 1, n_features)
    and X_train to (1, n_train, n_features), then subtracts.
    """
    diff = X_batch[:, np.newaxis, :] - X_train[np.newaxis, :, :]
    distances = np.sum(np.abs(diff), axis=2)

    return distances

# Quick test with small samples
test_distances = manhattan_distance_batch(X_test[:3], X_train[:5])
print(f"Distance matrix shape: {test_distances.shape}")
print(f"Sample distances from test[0] to first 5 train samples:")
print(test_distances[0])

Distance matrix shape: (3, 5)
Sample distances from test[0] to first 5 train samples:
[28.32663799 16.68044501 35.29420896 31.71431377 16.84263946]
